# Magnetic Structure Visualization with PyVista

Interactive 3D visualization of magnetic systems using PyVista.

This notebook demonstrates:
1. **Atomic structure visualization** - 3D crystal structure with type-colored atoms
2. **Magnetic moment vectors** - Spin orientations as arrows/cones
3. **Exchange network visualization** - Neighbor bonds colored by interaction type
4. **Interactive magnetic textures** - Skyrmions, domain walls, vortices
5. **Time evolution animation** - Spin dynamics visualization

**Requirements:**
```bash
pip install pyvista numpy matplotlib ase
```

In [ ]:
import numpy as np
import pyvista as pv
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb

# Set PyVista backend for notebook
pv.set_jupyter_backend('static')  # Use 'client' for interactive, 'static' for simple

## 1. Define Multi-Species Crystal Structure
Create the same Fe/Co/B system from the interactive setup notebook.

In [ ]:
# Lattice parameters
alat = 2.5  # Angstrom

# Normalized lattice vectors
lattice_vectors = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

# Basis atoms with types: [x, y, z, type]
basis_fractional = np.array([
    [0.0, 0.0, 0.0, 1],  # Fe
    [0.5, 0.5, 0.0, 2],  # Co
    [0.5, 0.0, 0.5, 2],  # Co
    [0.0, 0.5, 0.5, 3],  # B
])

# For visualization, use smaller system
n1, n2, n3 = 4, 4, 3
natom_basis = len(basis_fractional)
natom = natom_basis * n1 * n2 * n3

print(f"System: {natom} atoms in {n1}×{n2}×{n3} supercell")
print(f"Basis: {natom_basis} atoms per unit cell")

In [ ]:
def generate_structure(lattice, basis_frac, n1, n2, n3, scale):
    """Generate atomic coordinates, types, and initial moments."""
    positions = []
    types = []
    
    for i in range(n1):
        for j in range(n2):
            for k in range(n3):
                cell_origin = scale * (i * lattice[0] + j * lattice[1] + k * lattice[2])
                for atom_data in basis_frac:
                    atom_frac = atom_data[:3]
                    atom_type = int(atom_data[3])
                    atom_pos = scale * (atom_frac @ lattice) + cell_origin
                    positions.append(atom_pos)
                    types.append(atom_type)
    
    return np.array(positions), np.array(types)

# Generate structure
positions, atom_types = generate_structure(
    lattice_vectors, basis_fractional, n1, n2, n3, alat
)

print(f"Generated {len(positions)} atoms")
print(f"Position range: [{positions.min():.2f}, {positions.max():.2f}] Å")

## 2. Define Exchange Interactions

**This is where you customize the magnetic interactions!**

The exchange matrix defines how strongly different atom types interact at different distances.

### Format: `{(type_i, type_j, shell): J_value}`
- `type_i, type_j`: Atom types (1=Fe, 2=Co, 3=B)
- `shell`: Distance shell (1=NN, 2=NNN, etc.)
- `J_value`: Exchange strength in mRy (1 mRy ≈ 13.6 meV)

### How to modify interactions:
1. **Change J values**: Modify the numbers below to change interaction strength
   - Positive J → Ferromagnetic coupling (parallel spins)
   - Negative J → Antiferromagnetic coupling (antiparallel spins)
2. **Add new shells**: Define interactions for shell=3, 4, etc.
3. **Remove interactions**: Delete lines or set J=0

### Distance shells:
Define which neighbors belong to which shell based on distance (in units of alat).

In [ ]:
# Exchange parameters (mRy = milli-Rydberg)
# Note: 1 meV ≈ 0.0735 mRy, 1 mRy ≈ 13.6 meV
# 
# **MODIFY THESE VALUES** to change magnetic interactions:
#
exchange_matrix = {
    # === Nearest neighbor interactions (shell 1) ===
    # Distance: ~0.7*alat (e.g., [0.5, 0.5, 0.0])
    (1, 1, 1): 1.544,   # Fe-Fe NN (~21 meV) - ferromagnetic
    (2, 2, 1): 1.323,   # Co-Co NN (~18 meV) - ferromagnetic
    (3, 3, 1): 0.368,   # B-B NN (~5 meV) - weak ferromagnetic
    (1, 2, 1): 1.433,   # Fe-Co NN (~19.5 meV)
    (2, 1, 1): 1.433,   # Co-Fe NN (symmetric)
    (1, 3, 1): 0.588,   # Fe-B NN (~8 meV)
    (3, 1, 1): 0.588,   # B-Fe NN
    (2, 3, 1): 0.662,   # Co-B NN (~9 meV)
    (3, 2, 1): 0.662,   # B-Co NN
    
    # === Next-nearest neighbor interactions (shell 2) ===
    # Distance: ~1.0*alat (e.g., [1.0, 0.0, 0.0])
    (1, 1, 2): 0.5,     # Fe-Fe NNN - weaker interaction
    (2, 2, 2): 0.4,     # Co-Co NNN
    (1, 2, 2): 0.45,    # Fe-Co NNN
    (2, 1, 2): 0.45,    # Co-Fe NNN
    
    # === Add more shells here ===
    # (1, 1, 3): 0.2,   # Fe-Fe third neighbor
    # etc.
}

# Distance ranges for each shell (in units of alat)
# **MODIFY THESE** if you change the crystal structure
distance_shells = {
    1: (0.0, 0.9),      # NN: 0 < r < 0.9*alat
    2: (0.9, 1.3),      # NNN: 0.9*alat < r < 1.3*alat
    3: (1.3, 2.0),      # Further neighbors
}

print("=" * 60)
print("EXCHANGE INTERACTION MATRIX (mRy)")
print("=" * 60)
print("\nNearest Neighbors (NN):")
print("     Fe    Co    B")
for i in [1, 2, 3]:
    row = []
    for j in [1, 2, 3]:
        val = exchange_matrix.get((i, j, 1), 0.0)
        row.append(f"{val:5.3f}")
    label = ['Fe', 'Co', 'B'][i-1]
    print(f"{label}  " + "  ".join(row))

print("\nNext-Nearest Neighbors (NNN):")
print("     Fe    Co    B")
for i in [1, 2, 3]:
    row = []
    for j in [1, 2, 3]:
        val = exchange_matrix.get((i, j, 2), 0.0)
        row.append(f"{val:5.3f}")
    label = ['Fe', 'Co', 'B'][i-1]
    print(f"{label}  " + "  ".join(row))

print("\n" + "=" * 60)
print("TIP: Change values above and re-run this cell + section 3!")
print("=" * 60)

## 3. Generate Initial Magnetic Configuration
Create various magnetic textures for visualization.

**After defining interactions, choose an initial magnetic state:**
- Ferromagnetic (all spins up)
- Antiferromagnetic (alternating pattern)
- Vortex (in-plane rotation)
- Skyrmion (topological texture)

In [ ]:
def create_ferromagnetic(positions):
    """All spins point up (+z)."""
    moments = np.zeros((len(positions), 3))
    moments[:, 2] = 1.0
    return moments

def create_antiferromagnetic(positions, atom_types):
    """Alternating spin pattern."""
    moments = np.zeros((len(positions), 3))
    # Alternate based on position parity
    for i, pos in enumerate(positions):
        parity = int(np.sum(np.round(pos / alat))) % 2
        moments[i, 2] = 1.0 if parity == 0 else -1.0
    return moments

def create_vortex(positions, center=None):
    """Magnetic vortex in xy-plane."""
    if center is None:
        center = positions.mean(axis=0)
    
    moments = np.zeros((len(positions), 3))
    for i, pos in enumerate(positions):
        dx = pos[0] - center[0]
        dy = pos[1] - center[1]
        r = np.sqrt(dx**2 + dy**2)
        
        if r < 0.1:
            # Core points up
            moments[i] = [0, 0, 1]
        else:
            # Tangential direction
            moments[i] = [-dy/r, dx/r, 0]
    
    return moments

def create_skyrmion(positions, center=None, radius=5.0):
    """Néel skyrmion texture."""
    if center is None:
        center = positions.mean(axis=0)
    
    moments = np.zeros((len(positions), 3))
    for i, pos in enumerate(positions):
        dx = pos[0] - center[0]
        dy = pos[1] - center[1]
        r = np.sqrt(dx**2 + dy**2)
        
        # Skyrmion profile: theta(r)
        theta = np.pi * (1 - np.tanh((r - radius) / 2))
        phi = np.arctan2(dy, dx)
        
        moments[i, 0] = np.sin(theta) * np.cos(phi)
        moments[i, 1] = np.sin(theta) * np.sin(phi)
        moments[i, 2] = np.cos(theta)
    
    return moments

# Create different configurations
config_fm = create_ferromagnetic(positions)
config_afm = create_antiferromagnetic(positions, atom_types)
config_vortex = create_vortex(positions)
config_skyrmion = create_skyrmion(positions, radius=4.0)

# Select configuration for visualization
moments = config_skyrmion.copy()

print(f"✓ Created magnetic configuration")
print(f"  Mean magnetization: {moments.mean(axis=0)}")
print(f"  |M| per atom: {np.linalg.norm(moments, axis=1).mean():.3f}")
print(f"\n✓ Available configs (change 'moments = config_XXX' above):")
print(f"  - config_fm: Ferromagnetic (all up)")
print(f"  - config_afm: Antiferromagnetic (checkerboard)")
print(f"  - config_vortex: In-plane vortex")
print(f"  - config_skyrmion: Néel skyrmion")
print(f"\nℹ️  You can also load relaxed configurations from UppASD (see section 11)")

## 3. Build Exchange Network with ASE

This section automatically finds all neighbor bonds based on the exchange matrix you defined.
The bonds will be visualized to show the interaction network.

In [ ]:
from ase import Atoms
from ase.neighborlist import neighbor_list

def build_neighbor_bonds(positions, atom_types, cutoff=3.5):
    """
    Find all neighbor pairs within cutoff.
    
    Returns:
    --------
    bonds : list of tuples
        Each tuple: (i, j, r_ij, type_i, type_j, distance)
    """
    # Create ASE atoms object
    # Use positions directly (already in Cartesian)
    box_size = positions.max(axis=0) + alat
    atoms = Atoms(
        numbers=atom_types,
        positions=positions,
        cell=box_size,
        pbc=False  # Open boundaries for finite cluster
    )
    
    # Find neighbors
    i_indices, j_indices, distances = neighbor_list('ijd', atoms, cutoff)
    
    bonds = []
    seen = set()
    
    for idx in range(len(i_indices)):
        i = i_indices[idx]
        j = j_indices[idx]
        dist = distances[idx]
        
        # Skip self-pairs
        if i == j:
            continue
        
        # Store only one direction (i < j)
        if i > j:
            i, j = j, i
        
        if (i, j) in seen:
            continue
        
        seen.add((i, j))
        r_ij = positions[j] - positions[i]
        
        bonds.append((i, j, r_ij, atom_types[i], atom_types[j], dist))
    
    return bonds

# Build neighbor network
bonds = build_neighbor_bonds(positions, atom_types, cutoff=3.0)

print(f"✓ Found {len(bonds)} neighbor bonds")

# Count by interaction type
from collections import Counter
type_pairs = Counter([(ti, tj) for (_, _, _, ti, tj, _) in bonds])
print(f"\nBond types:")
type_labels = {1: 'Fe', 2: 'Co', 3: 'B'}
for (ti, tj), count in sorted(type_pairs.items()):
    print(f"  {type_labels[ti]}-{type_labels[tj]}: {count} bonds")

## 4. Visualize Atomic Structure
3D plot of crystal structure with type-colored atoms.

In [ ]:
def plot_structure(positions, atom_types, show_bonds=False, bonds=None):
    """
    Visualize atomic structure with PyVista.
    """
    plotter = pv.Plotter()
    
    # Define colors for each atom type
    type_colors = {
        1: 'red',      # Fe
        2: 'blue',     # Co
        3: 'green'     # B
    }
    
    # Define sizes for each atom type
    type_radii = {
        1: 0.3,  # Fe
        2: 0.25, # Co
        3: 0.2   # B
    }
    
    # Plot atoms by type
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        
        pos_type = positions[mask]
        
        # Create point cloud
        cloud = pv.PolyData(pos_type)
        
        # Add spheres
        spheres = cloud.glyph(
            geom=pv.Sphere(radius=type_radii[atype]),
            scale=False
        )
        
        type_name = {1: 'Fe', 2: 'Co', 3: 'B'}[atype]
        plotter.add_mesh(
            spheres,
            color=type_colors[atype],
            label=type_name,
            smooth_shading=True
        )
    
    # Plot bonds if requested
    if show_bonds and bonds is not None:
        for (i, j, r_ij, ti, tj, dist) in bonds:
            line = pv.Line(positions[i], positions[j])
            plotter.add_mesh(line, color='gray', line_width=1, opacity=0.3)
    
    # Set view
    plotter.add_legend()
    plotter.set_background('white')
    plotter.add_axes()
    plotter.view_isometric()
    
    return plotter

# Create visualization
pl = plot_structure(positions, atom_types, show_bonds=True, bonds=bonds)
pl.show()

## 5. Relax Magnetic Structure with UppASD

**This step runs a spin dynamics relaxation with your custom exchange interactions!**

The initial magnetic configuration (from Section 3) will be relaxed using:
- Monte Carlo annealing to find low-energy states
- Spin dynamics with damping to reach equilibrium

After this section, all visualizations will show the **relaxed** magnetic structure.

**To investigate different interactions:**
1. Modify `exchange_matrix` in Section 2
2. Re-run Sections 2, 3, 5 (this section)
3. View the relaxed structure in Sections 6-10

In [ ]:
from ase import Atoms
from ase.neighborlist import neighbor_list

def build_exchange_list(lattice_vectors, basis_positions, basis_types, 
                        exchange_matrix, distance_shells, cutoff=4.0, scale=1.0):
    """Build UppASD exchange list from exchange matrix."""
    cell_scaled = lattice_vectors * scale
    atoms = Atoms(
        numbers=basis_types,
        positions=basis_positions @ cell_scaled,
        cell=cell_scaled,
        pbc=True
    )
    
    i_indices, j_indices, offsets = neighbor_list('ijS', atoms, cutoff)
    exchange_dict = {}
    
    for idx in range(len(i_indices)):
        i = i_indices[idx]
        j = j_indices[idx]
        offset = offsets[idx]
        
        type_i = basis_types[i]
        type_j = basis_types[j]
        
        r_i = atoms.positions[i]
        r_j = atoms.positions[j] + offset @ cell_scaled
        r_ij_cart = r_j - r_i
        r_ij_frac = r_ij_cart / scale
        
        dist = np.linalg.norm(r_ij_frac)
        if dist < 1e-6:
            continue
        
        shell = None
        for s, (d_min, d_max) in distance_shells.items():
            if d_min < dist <= d_max:
                shell = s
                break
        
        if shell is None or (type_i, type_j, shell) not in exchange_matrix:
            continue
        
        J_ij = exchange_matrix[(type_i, type_j, shell)]
        r_ij_tuple = tuple(np.round(r_ij_frac, decimals=6))
        
        key = (type_i, type_j, r_ij_tuple)
        if key not in exchange_dict:
            exchange_dict[key] = J_ij
    
    exchange_list = []
    for (type_i, type_j, r_ij_tuple), J_ij in sorted(exchange_dict.items()):
        exchange_list.append((type_i, type_j, r_ij_tuple[0], r_ij_tuple[1], r_ij_tuple[2], J_ij))
    
    return exchange_list

# Build exchange list for UppASD
exchange_list = build_exchange_list(
    lattice_vectors,
    basis_fractional[:, :3],
    basis_fractional[:, 3].astype(int),
    exchange_matrix,
    distance_shells,
    cutoff=4.0,
    scale=alat
)

print(f"✓ Built exchange list: {len(exchange_list)} unique interactions")
print(f"  (Using exchange matrix from Section 2)")

In [ ]:
import os

# Create simulation directory
sim_dir = Path('./relax_sim')
sim_dir.mkdir(exist_ok=True)

# Write inpsd.dat
inpsd_content = f"""simid  relaxation
ncell  {n1}  {n2}  {n3}
BC  P  P  P
cell
1.0  0.0  0.0
0.0  1.0  0.0
0.0  0.0  1.0
Sym  0
alat  {alat:.6f}

posfile  ./posfile.dat
momfile  ./momfile.dat
exchange  ./jfile.dat

do_prnstruct  1

Mensemble  1
Initmag  3

# Initial relaxation with Monte Carlo
ip_mode  M
ip_mcanneal  1
2000 100

# Spin dynamics relaxation
mode  S
temp  50
damping  0.1
timestep  1.0e-16
nstep  2000

do_avrg  Y
avrg_step  100
"""

with open(sim_dir / 'inpsd.dat', 'w') as f:
    f.write(inpsd_content)

# Write posfile.dat
with open(sim_dir / 'posfile.dat', 'w') as f:
    for i, pos in enumerate(basis_fractional[:, :3], 1):
        atype = int(basis_fractional[i-1, 3])
        f.write(f"{i}  {atype}  {pos[0]:.6f}  {pos[1]:.6f}  {pos[2]:.6f}\n")

# Write momfile.dat (species-dependent moments)
moments_per_type = {1: 2.2, 2: 1.7, 3: 0.5}
with open(sim_dir / 'momfile.dat', 'w') as f:
    for i, atom_data in enumerate(basis_fractional, 1):
        atype = int(atom_data[3])
        mag = moments_per_type[atype]
        f.write(f"{i}  1  {mag:.6f}  0.0 0.0 1.0\n")

# Write jfile.dat with custom exchange matrix
with open(sim_dir / 'jfile.dat', 'w') as f:
    for (type_i, type_j, rx, ry, rz, J_ij) in exchange_list:
        f.write(f"{type_i} {type_j}  {rx:.6f}  {ry:.6f}  {rz:.6f}  {J_ij:.6f}\n")

print("✓ Generated UppASD input files in:", sim_dir)
print(f"  • inpsd.dat: MC + SD relaxation")
print(f"  • posfile.dat: {len(basis_fractional)} basis atoms")
print(f"  • momfile.dat: Species-dependent moments")
print(f"  • jfile.dat: {len(exchange_list)} exchange interactions")

In [ ]:
# Run UppASD relaxation
print("=" * 60)
print("RUNNING UPPASD RELAXATION")
print("=" * 60)
print(f"Initial configuration: {moments.shape[0]} spins")
print(f"Using exchange matrix from Section 2")
print("-" * 60)

original_dir = os.getcwd()
os.chdir(sim_dir)

try:
    from uppasd import Simulator
    
    with Simulator() as sim:
        print(f"\n✓ Initialized: {sim.natom} atoms, {sim.mensemble} ensembles")
        
        # Run relaxation
        print("\nRunning relaxation (MC + spin dynamics)...")
        final_moments = sim.relax(
            mode='S',
            temperature=50,
            steps=2000,
            timestep=1e-16,
            damping=0.1
        )
        
        print(f"\n✓ Relaxation complete!")
        print(f"  Final energy: {sim.energy:.3f}")
        
        # Extract relaxed moments for full supercell
        moments = final_moments[:, :, 0].T  # Shape: (natom, 3)
        
        # Save for later use
        np.save('moments_relaxed.npy', moments)
        
        print(f"\n✓ Relaxed magnetic structure:")
        print(f"  Mean magnetization: {moments.mean(axis=0)}")
        print(f"  |M| per atom: {np.linalg.norm(moments, axis=1).mean():.3f}")
        
except ImportError:
    print("\n⚠️  UppASD Python module not found!")
    print("Using initial configuration instead of relaxed.")
    print("\nTo enable relaxation, install UppASD:")
    print("  pip install -e /path/to/UppASD")
    # Keep using the initial moments from Section 3
    
finally:
    os.chdir(original_dir)

print("\n" + "=" * 60)
print("✨ Magnetic structure ready for visualization!")
print("=" * 60)

## 6. Visualize Relaxed Magnetic Structure

**This shows the RELAXED spin configuration from Section 5.**

Spin orientations are displayed as arrows, colored by their z-component (Mz).

In [ ]:
def plot_magnetic_structure(positions, moments, atom_types, 
                           arrow_scale=0.8, show_atoms=True):
    """
    Visualize magnetic structure with spin arrows.
    """
    plotter = pv.Plotter()
    
    # Atom colors
    type_colors = {1: 'red', 2: 'blue', 3: 'green'}
    type_radii = {1: 0.25, 2: 0.2, 3: 0.15}
    
    # Plot atoms (semi-transparent)
    if show_atoms:
        for atype in [1, 2, 3]:
            mask = atom_types == atype
            if not np.any(mask):
                continue
            
            cloud = pv.PolyData(positions[mask])
            spheres = cloud.glyph(
                geom=pv.Sphere(radius=type_radii[atype]),
                scale=False
            )
            plotter.add_mesh(
                spheres,
                color=type_colors[atype],
                opacity=0.3,
                smooth_shading=True
            )
    
    # Create arrows for magnetic moments
    # Scale arrows by moment magnitude
    moment_mags = np.linalg.norm(moments, axis=1)
    
    # Create point cloud with moment data
    cloud = pv.PolyData(positions)
    cloud['moments'] = moments
    cloud['mz'] = moments[:, 2]  # z-component for coloring
    
    # Create arrows
    arrows = cloud.glyph(
        orient='moments',
        scale='moments',
        factor=arrow_scale,
        geom=pv.Arrow()
    )
    
    # Color by z-component
    plotter.add_mesh(
        arrows,
        scalars='mz',
        cmap='coolwarm',
        clim=[-1, 1],
        show_scalar_bar=True,
        scalar_bar_args={'title': 'Mz'},
        smooth_shading=True
    )
    
    plotter.set_background('white')
    plotter.add_axes()
    plotter.view_isometric()
    
    return plotter

# Visualize relaxed magnetic structure
pl = plot_magnetic_structure(positions, moments, atom_types, arrow_scale=0.7)
pl.show()

## 7. Exchange Network Visualization
Show neighbor bonds colored by interaction type and strength.

In [ ]:
def plot_exchange_network(positions, atom_types, bonds, moments=None):
    """
    Visualize exchange interaction network.
    
    Bonds are colored by interaction type.
    """
    plotter = pv.Plotter()
    
    # Atom colors
    type_colors = {1: 'red', 2: 'blue', 3: 'green'}
    type_radii = {1: 0.2, 2: 0.15, 3: 0.12}
    
    # Plot atoms
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        
        cloud = pv.PolyData(positions[mask])
        spheres = cloud.glyph(
            geom=pv.Sphere(radius=type_radii[atype]),
            scale=False
        )
        plotter.add_mesh(
            spheres,
            color=type_colors[atype],
            smooth_shading=True
        )
    
    # Define bond colors by type pair
    bond_colors = {
        (1, 1): 'darkred',
        (2, 2): 'darkblue',
        (3, 3): 'darkgreen',
        (1, 2): 'purple',
        (2, 1): 'purple',
        (1, 3): 'orange',
        (3, 1): 'orange',
        (2, 3): 'cyan',
        (3, 2): 'cyan',
    }
    
    # Group bonds by type
    from collections import defaultdict
    bonds_by_type = defaultdict(list)
    for bond in bonds:
        i, j, r_ij, ti, tj, dist = bond
        bonds_by_type[(ti, tj)].append((i, j))
    
    # Plot bonds by type
    type_labels = {1: 'Fe', 2: 'Co', 3: 'B'}
    for (ti, tj), bond_list in bonds_by_type.items():
        color = bond_colors.get((ti, tj), 'gray')
        label = f"{type_labels[ti]}-{type_labels[tj]}"
        
        for (i, j) in bond_list:
            line = pv.Line(positions[i], positions[j])
            plotter.add_mesh(
                line,
                color=color,
                line_width=2,
                label=label if (i, j) == bond_list[0] else None
            )
    
    # Add magnetic moments if provided
    if moments is not None:
        cloud = pv.PolyData(positions)
        cloud['moments'] = moments
        arrows = cloud.glyph(
            orient='moments',
            scale='moments',
            factor=0.5,
            geom=pv.Arrow()
        )
        plotter.add_mesh(arrows, color='black', opacity=0.5)
    
    plotter.add_legend()
    plotter.set_background('white')
    plotter.add_axes()
    plotter.view_isometric()
    
    return plotter

# Visualize exchange network
pl = plot_exchange_network(positions, atom_types, bonds, moments=moments)
pl.show()

## 8. Magnetic Texture Close-up
Zoom in on interesting regions (e.g., skyrmion core).

In [ ]:
def plot_texture_closeup(positions, moments, atom_types, 
                        region_center=None, region_size=8.0):
    """
    Visualize magnetic texture in a selected region.
    """
    if region_center is None:
        region_center = positions.mean(axis=0)
    
    # Select atoms in region
    distances = np.linalg.norm(positions - region_center, axis=1)
    mask = distances < region_size
    
    pos_region = positions[mask]
    mom_region = moments[mask]
    types_region = atom_types[mask]
    
    print(f"Showing {len(pos_region)} atoms in region")
    
    plotter = pv.Plotter()
    
    # Plot atoms with type colors
    type_colors = {1: 'red', 2: 'blue', 3: 'green'}
    for atype in [1, 2, 3]:
        mask_type = types_region == atype
        if not np.any(mask_type):
            continue
        
        cloud = pv.PolyData(pos_region[mask_type])
        spheres = cloud.glyph(
            geom=pv.Sphere(radius=0.3),
            scale=False
        )
        plotter.add_mesh(
            spheres,
            color=type_colors[atype],
            opacity=0.4,
            smooth_shading=True
        )
    
    # Add moment arrows
    cloud = pv.PolyData(pos_region)
    cloud['moments'] = mom_region
    cloud['mz'] = mom_region[:, 2]
    
    arrows = cloud.glyph(
        orient='moments',
        scale='moments',
        factor=1.0,
        geom=pv.Arrow()
    )
    
    plotter.add_mesh(
        arrows,
        scalars='mz',
        cmap='coolwarm',
        clim=[-1, 1],
        show_scalar_bar=True,
        scalar_bar_args={'title': 'Mz', 'position_x': 0.85},
        smooth_shading=True
    )
    
    plotter.set_background('white')
    plotter.add_axes()
    plotter.camera_position = 'xy'
    plotter.camera.zoom(1.5)
    
    return plotter

# Visualize skyrmion core
pl = plot_texture_closeup(positions, moments, atom_types, region_size=6.0)
pl.show()

## 9. Multiple Views: Combined Visualization
Show structure, moments, and exchange network side-by-side.

In [ ]:
def plot_multi_view(positions, moments, atom_types, bonds):
    """
    Create multi-panel visualization.
    """
    plotter = pv.Plotter(shape=(1, 3))
    
    type_colors = {1: 'red', 2: 'blue', 3: 'green'}
    type_radii = {1: 0.25, 2: 0.2, 3: 0.15}
    
    # Panel 1: Structure only
    plotter.subplot(0, 0)
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        cloud = pv.PolyData(positions[mask])
        spheres = cloud.glyph(geom=pv.Sphere(radius=type_radii[atype]), scale=False)
        plotter.add_mesh(spheres, color=type_colors[atype], smooth_shading=True)
    plotter.add_text("Atomic Structure", font_size=12)
    plotter.view_isometric()
    
    # Panel 2: Magnetic moments
    plotter.subplot(0, 1)
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        cloud = pv.PolyData(positions[mask])
        spheres = cloud.glyph(geom=pv.Sphere(radius=type_radii[atype]), scale=False)
        plotter.add_mesh(spheres, color=type_colors[atype], opacity=0.3)
    
    cloud = pv.PolyData(positions)
    cloud['moments'] = moments
    cloud['mz'] = moments[:, 2]
    arrows = cloud.glyph(orient='moments', scale='moments', factor=0.7, geom=pv.Arrow())
    plotter.add_mesh(arrows, scalars='mz', cmap='coolwarm', clim=[-1, 1])
    plotter.add_text("Magnetic Moments", font_size=12)
    plotter.view_isometric()
    
    # Panel 3: Exchange network
    plotter.subplot(0, 2)
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        cloud = pv.PolyData(positions[mask])
        spheres = cloud.glyph(geom=pv.Sphere(radius=type_radii[atype]), scale=False)
        plotter.add_mesh(spheres, color=type_colors[atype], smooth_shading=True)
    
    for (i, j, r_ij, ti, tj, dist) in bonds:
        line = pv.Line(positions[i], positions[j])
        plotter.add_mesh(line, color='gray', line_width=1, opacity=0.5)
    
    plotter.add_text("Exchange Network", font_size=12)
    plotter.view_isometric()
    
    plotter.set_background('white')
    plotter.link_views()
    
    return plotter

# Create multi-view
pl = plot_multi_view(positions, moments, atom_types, bonds)
pl.show()

## 10. Export Visualizations
Save high-quality images for publications.

In [ ]:
# Create output directory
output_dir = Path('./visualization_output')
output_dir.mkdir(exist_ok=True)

# Save different views
print("Generating high-resolution images...")

# 1. Magnetic structure
pl = plot_magnetic_structure(positions, moments, atom_types, arrow_scale=0.7)
pl.screenshot(str(output_dir / 'magnetic_structure.png'), scale=3)
pl.close()
print("✓ Saved: magnetic_structure.png")

# 2. Exchange network
pl = plot_exchange_network(positions, atom_types, bonds[:200])  # Subset for clarity
pl.screenshot(str(output_dir / 'exchange_network.png'), scale=3)
pl.close()
print("✓ Saved: exchange_network.png")

# 3. Texture closeup
pl = plot_texture_closeup(positions, moments, atom_types, region_size=6.0)
pl.screenshot(str(output_dir / 'texture_closeup.png'), scale=3)
pl.close()
print("✓ Saved: texture_closeup.png")

print(f"\n✓ All images saved to {output_dir}/")

## 10. Interactive Exploration
Use PyVista's interactive mode (requires local Jupyter or client backend).

In [ ]:
# Uncomment to enable interactive mode
# pv.set_jupyter_backend('client')

# Create interactive plotter
plotter = pv.Plotter()

# Add magnetic structure
type_colors = {1: 'red', 2: 'blue', 3: 'green'}
for atype in [1, 2, 3]:
    mask = atom_types == atype
    if not np.any(mask):
        continue
    cloud = pv.PolyData(positions[mask])
    spheres = cloud.glyph(geom=pv.Sphere(radius=0.25), scale=False)
    plotter.add_mesh(spheres, color=type_colors[atype], opacity=0.4)

cloud = pv.PolyData(positions)
cloud['moments'] = moments
cloud['mz'] = moments[:, 2]
arrows = cloud.glyph(orient='moments', scale='moments', factor=0.7, geom=pv.Arrow())
plotter.add_mesh(arrows, scalars='mz', cmap='coolwarm', clim=[-1, 1], show_scalar_bar=True)

plotter.set_background('white')
plotter.add_axes()
plotter.show()

print("\nInteractive controls:")
print("  - Left mouse: Rotate")
print("  - Middle mouse: Pan")
print("  - Right mouse / scroll: Zoom")
print("  - 'r': Reset camera")

## Summary

This notebook demonstrated:
- ✓ **3D structure visualization** with type-colored atoms
- ✓ **Customizable exchange interactions** defined in Section 2
- ✓ **UppASD relaxation** integrated into the workflow (Section 5)
- ✓ **Relaxed magnetic structure visualization** with arrows colored by orientation
- ✓ **Exchange network mapping** showing neighbor bonds by type
- ✓ **Texture close-ups** for detailed regions
- ✓ **Multi-panel comparisons** for comprehensive analysis
- ✓ **High-quality exports** for publications
- ✓ **Interactive 3D exploration** with PyVista

**Complete workflow:**
1. Define crystal structure (Section 1)
2. **Customize exchange interactions** (Section 2) ← Modify here!
3. Choose initial magnetic state (Section 3)
4. Visualize atomic structure (Section 4)
5. **Run UppASD relaxation** (Section 5) ← Relaxes with your interactions!
6. Visualize relaxed magnetic structure (Sections 6-10)

**To investigate different interactions:**
- Modify `exchange_matrix` in Section 2
- Re-run Sections 2, 3, 5, 6
- Compare the relaxed structures!

**Examples to try:**
- Antiferromagnetic interactions: Change positive J to negative
- Longer-range interactions: Add shell 3, 4 interactions
- Different initial states: FM, AFM, vortex, skyrmion (Section 3)
- Competing interactions: Mix FM and AFM in different shells

**Next steps:**
- Load time-dependent trajectories for animations
- Analyze topological properties (skyrmion number, chirality)
- Add temperature dependence studies
- Measure correlation functions and order parameters

In [ ]:
from ase import Atoms
from ase.neighborlist import neighbor_list

def build_exchange_list(lattice_vectors, basis_positions, basis_types, 
                        exchange_matrix, distance_shells, cutoff=4.0, scale=1.0):
    """
    Build UppASD exchange list from exchange matrix.
    Same function as in interactive_setup_notebook.ipynb.
    """
    cell_scaled = lattice_vectors * scale
    atoms = Atoms(
        numbers=basis_types,
        positions=basis_positions @ cell_scaled,
        cell=cell_scaled,
        pbc=True
    )
    
    i_indices, j_indices, offsets = neighbor_list('ijS', atoms, cutoff)
    exchange_dict = {}
    
    for idx in range(len(i_indices)):
        i = i_indices[idx]
        j = j_indices[idx]
        offset = offsets[idx]
        
        type_i = basis_types[i]
        type_j = basis_types[j]
        
        r_i = atoms.positions[i]
        r_j = atoms.positions[j] + offset @ cell_scaled
        r_ij_cart = r_j - r_i
        r_ij_frac = r_ij_cart / scale
        
        dist = np.linalg.norm(r_ij_frac)
        if dist < 1e-6:
            continue
        
        shell = None
        for s, (d_min, d_max) in distance_shells.items():
            if d_min < dist <= d_max:
                shell = s
                break
        
        if shell is None or (type_i, type_j, shell) not in exchange_matrix:
            continue
        
        J_ij = exchange_matrix[(type_i, type_j, shell)]
        r_ij_tuple = tuple(np.round(r_ij_frac, decimals=6))
        
        key = (type_i, type_j, r_ij_tuple)
        if key not in exchange_dict:
            exchange_dict[key] = J_ij
    
    exchange_list = []
    for (type_i, type_j, r_ij_tuple), J_ij in sorted(exchange_dict.items()):
        exchange_list.append((type_i, type_j, r_ij_tuple[0], r_ij_tuple[1], r_ij_tuple[2], J_ij))
    
    return exchange_list

# Build exchange list
exchange_list = build_exchange_list(
    lattice_vectors,
    basis_fractional[:, :3],
    basis_fractional[:, 3].astype(int),
    exchange_matrix,
    distance_shells,
    cutoff=4.0,
    scale=alat
)

print(f"✓ Built exchange list: {len(exchange_list)} unique interactions")

In [ ]:
import os

# Create simulation directory
sim_dir = Path('./viz_sim')
sim_dir.mkdir(exist_ok=True)

# Write inpsd.dat
inpsd_content = f"""simid  viz_relaxation
ncell  {n1}  {n2}  {n3}
BC  P  P  P
cell
1.0  0.0  0.0
0.0  1.0  0.0
0.0  0.0  1.0
Sym  0
alat  {alat:.6f}

posfile  ./posfile.dat
momfile  ./momfile.dat
exchange  ./jfile.dat

do_prnstruct  1

Mensemble  1
Initmag  3

# Initial relaxation with Monte Carlo
ip_mode  M
ip_mcanneal  1
2000 100

# Spin dynamics relaxation
mode  S
temp  50
damping  0.1
timestep  1.0e-16
nstep  3000

# Output final structure
do_avrg  Y
avrg_step  100
do_prn_ams  Y
"""

with open(sim_dir / 'inpsd.dat', 'w') as f:
    f.write(inpsd_content)

# Write posfile.dat
with open(sim_dir / 'posfile.dat', 'w') as f:
    for i, pos in enumerate(basis_fractional[:, :3], 1):
        atype = int(basis_fractional[i-1, 3])
        f.write(f"{i}  {atype}  {pos[0]:.6f}  {pos[1]:.6f}  {pos[2]:.6f}\n")

# Write momfile.dat (species-dependent moments)
moments_per_type = {1: 2.2, 2: 1.7, 3: 0.5}
with open(sim_dir / 'momfile.dat', 'w') as f:
    for i, atom_data in enumerate(basis_fractional, 1):
        atype = int(atom_data[3])
        mag = moments_per_type[atype]
        f.write(f"{i}  1  {mag:.6f}  0.0 0.0 1.0\n")

# Write jfile.dat with your custom exchange matrix
with open(sim_dir / 'jfile.dat', 'w') as f:
    for (type_i, type_j, rx, ry, rz, J_ij) in exchange_list:
        f.write(f"{type_i} {type_j}  {rx:.6f}  {ry:.6f}  {rz:.6f}  {J_ij:.6f}\n")

print("✓ Created UppASD input files in:", sim_dir)
print("\nFiles created:")
print(f"  - inpsd.dat: {len(open(sim_dir / 'inpsd.dat').readlines())} lines")
print(f"  - posfile.dat: {len(basis_fractional)} basis atoms")
print(f"  - momfile.dat: Species-dependent moments")
print(f"  - jfile.dat: {len(exchange_list)} exchange interactions")
print("\n" + "=" * 60)
print("READY TO RUN SIMULATION!")
print("=" * 60)

In [ ]:
# Run UppASD simulation
print("Running UppASD relaxation...")
print("This will use your custom exchange matrix from Section 2")
print("-" * 60)

original_dir = os.getcwd()
os.chdir(sim_dir)

try:
    from uppasd import Simulator
    
    with Simulator() as sim:
        print(f"\n✓ Initialized: {sim.natom} atoms")
        
        # Run relaxation
        final_moments = sim.relax(
            mode='S',
            temperature=50,
            steps=3000,
            timestep=1e-16,
            damping=0.1
        )
        
        print(f"\n✓ Relaxation complete!")
        print(f"  Final energy: {sim.energy:.3f}")
        
        # Save relaxed configuration
        moments_relaxed = final_moments[:, :, 0].T  # Shape: (natom, 3)
        np.save('moments_relaxed.npy', moments_relaxed)
        print(f"  Saved: moments_relaxed.npy")
        
except ImportError:
    print("\n⚠️  UppASD Python module not found!")
    print("Install with: pip install -e /path/to/UppASD")
    print("\nAlternatively, run from terminal:")
    print(f"  cd {sim_dir}")
    print("  sd < inpsd.dat")
    moments_relaxed = None
    
finally:
    os.chdir(original_dir)

print("\n" + "=" * 60)

In [ ]:
# Load and visualize relaxed structure
if moments_relaxed is None and (sim_dir / 'moments_relaxed.npy').exists():
    moments_relaxed = np.load(sim_dir / 'moments_relaxed.npy')
    print("✓ Loaded relaxed configuration from file")

if moments_relaxed is not None:
    print(f"\n✓ Relaxed magnetic configuration:")
    print(f"  Mean magnetization: {moments_relaxed.mean(axis=0)}")
    print(f"  |M| per atom: {np.linalg.norm(moments_relaxed, axis=1).mean():.3f}")
    
    # Visualize initial vs relaxed
    plotter = pv.Plotter(shape=(1, 2))
    type_colors = {1: 'red', 2: 'blue', 3: 'green'}
    
    # Left: Initial configuration
    plotter.subplot(0, 0)
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        cloud = pv.PolyData(positions[mask])
        spheres = cloud.glyph(geom=pv.Sphere(radius=0.2), scale=False)
        plotter.add_mesh(spheres, color=type_colors[atype], opacity=0.3)
    
    cloud = pv.PolyData(positions)
    cloud['moments'] = moments
    cloud['mz'] = moments[:, 2]
    arrows = cloud.glyph(orient='moments', scale='moments', factor=0.7, geom=pv.Arrow())
    plotter.add_mesh(arrows, scalars='mz', cmap='coolwarm', clim=[-1, 1])
    plotter.add_text("Initial State", font_size=12)
    plotter.view_isometric()
    
    # Right: Relaxed configuration
    plotter.subplot(0, 1)
    for atype in [1, 2, 3]:
        mask = atom_types == atype
        if not np.any(mask):
            continue
        cloud = pv.PolyData(positions[mask])
        spheres = cloud.glyph(geom=pv.Sphere(radius=0.2), scale=False)
        plotter.add_mesh(spheres, color=type_colors[atype], opacity=0.3)
    
    cloud = pv.PolyData(positions)
    cloud['moments'] = moments_relaxed
    cloud['mz'] = moments_relaxed[:, 2]
    arrows = cloud.glyph(orient='moments', scale='moments', factor=0.7, geom=pv.Arrow())
    plotter.add_mesh(arrows, scalars='mz', cmap='coolwarm', clim=[-1, 1], show_scalar_bar=True)
    plotter.add_text("Relaxed State", font_size=12)
    plotter.view_isometric()
    
    plotter.set_background('white')
    plotter.link_views()
    plotter.show()
    
    print("\n✨ Visualization complete!")
    print("\n💡 TIP: To investigate different interactions:")
    print("   1. Go back to Section 2")
    print("   2. Modify exchange_matrix values")
    print("   3. Re-run sections 2, 11")
    print("   4. Compare the relaxed magnetic structures!")
else:
    print("\n⚠️  No relaxed configuration available yet")
    print("Run the previous cell to perform relaxation")

## Summary

This notebook demonstrated:

**This section shows how to run UppASD simulations with your custom exchange interactions!**

After modifying the exchange matrix in Section 2, follow these steps:
1. Generate UppASD input files
2. Run relaxation simulation
3. Load and visualize the relaxed magnetic structure

This lets you investigate how different exchange interactions affect the final magnetic state.